In [ ]:
#r "nuget:HtmlAgilityPack,1.11.24"
#r "nuget:Microsoft.Data.Analysis,0.4.0"
#r "nuget:CsvHelper,15.0.5"

In [ ]:
using System;
using System.IO;
using System.Linq;
using System.Net;
using System.Text;
using HtmlAgilityPack;
using Microsoft.Data.Analysis;
using XPlot.Plotly;
Encoding.RegisterProvider(CodePagesEncodingProvider.Instance);

In [ ]:
class Uik
{
    public string Name { get; set; }
    public List<double> Data { get; set; }
    public DateTime ProtocolTime { get; set; }    
    
    public Uik(string name, List<double> data, DateTime protocolTime)
    {
        Name = name;
        Data = data;
        ProtocolTime = protocolTime;
    }
}

In [ ]:
var url1 = "https://www.cvk.gov.ua/pls/vp2019/wp335pt001f01=719.html";
var url2 = "https://www.cvk.gov.ua/pls/vp2019/wp335pt001f01=720.html";

In [ ]:
var request = WebRequest.CreateHttp(url1);
var webResponse = request.GetResponse();
var doc = new HtmlDocument();
using (var responseStream = webResponse.GetResponseStream())
    doc.Load(responseStream, Encoding.GetEncoding("windows-1251"));

In [ ]:
var links = doc.DocumentNode.SelectNodes("//table/tr/td[2]/a");
var hrefs = links.Select(node => "https://www.cvk.gov.ua/pls/vp2019/" + node.Attributes["href"].Value);

In [ ]:
static List<String> GetHeader(string url)
{
    var tikRequest = WebRequest.CreateHttp(url);
    var tikResponse = tikRequest.GetResponse();
    var tikDocument = new HtmlDocument();
    using (var responseStream = tikResponse.GetResponseStream())
        tikDocument.Load(responseStream, Encoding.GetEncoding("windows-1251"));
    var header = tikDocument.DocumentNode
        .SelectNodes("//table/thead/tr")
        .First()
        .SelectNodes("th")
        .Select(node => node.InnerHtml.Replace("<br>", " "))
        .ToList();
    return header;
}

In [ ]:
List<Uik> GetUiks(string url)
{
    var tikRequest = WebRequest.CreateHttp(url);
    var tikResponse = tikRequest.GetResponse();
    var tikDocument = new HtmlDocument();
    using (var responseStream = tikResponse.GetResponseStream())
        tikDocument.Load(responseStream, Encoding.GetEncoding("windows-1251"));
    var header = tikDocument.DocumentNode
        .SelectNodes("//table/thead/tr")
        .First()
        .SelectNodes("th")
        .Select(node => node.InnerHtml.Replace("<br>", " "));
    var uik = tikDocument.DocumentNode.SelectNodes("//table/tr")
        .Select(row => row.SelectNodes("td")
            .Select(node => node.InnerHtml.Replace("<b>", "")
                .Replace("</b>", ""))
            .ToList())
        .Select(row => new Uik(row.First(),
            row.Skip(1)
                .SkipLast(1)
                .Select(i => double.Parse(i))
                .ToList(),
            DateTime.Parse(row.Last())))
        .ToList();
    return uik;
}

In [ ]:
var header = GetHeader(hrefs.First()).Skip(1).ToList();
header[49] = "Явка";
var uiks = hrefs.Take(20)
    .SelectMany(url => GetUiks(url))
    .Select(uik =>
    {
        uik.Data = uik.Data.Take(8)
            .Concat(uik.Data.Skip(8)
                .Select(i => i / uik.Data[8]))
            .Concat(new [] {uik.Data[8] / uik.Data[1]})
            .ToList();
        return uik;
    })
    .ToList();

In [ ]:
header

# Рисуем графики

In [ ]:
Graph.Scatter GetScatter(IEnumerable<Uik> uiks, List<string> header, int candidateIndex, double markerSize) =>
    new Graph.Scatter()
    {
        x = uiks.Select(uik => uik.Data[49]),
        y = uiks.Select(uik => uik.Data[candidateIndex]),
        mode = "markers",
        marker = new Graph.Marker() { size = markerSize },
        name = header[candidateIndex]
    };

In [ ]:
var plot = Chart.Plot(new [] {23, 39}.Select(index => GetScatter(uiks, header, index, 1.5)));
var layout = new Layout.Layout
{
    xaxis = new Graph.Xaxis {title = "Явка избирателей"},
    yaxis = new Graph.Yaxis {title = "Процент голосов за кандидата"}
};
plot.WithLayout(layout);
plot.WithTitle("Результат кандидата");
display(plot);